# Alguns exemplos de código pyspark / SparkSQL

## Trabalhando com timezone / timestamp

Como trabalhar com campos não UTC (timezone incorporado)

```sql
-- forma correta, converte a constante para o timestamp de S.P.
/* The default format of the Spark Timestamp is yyyy-MM-dd HH:mm:ss.SSSS  */
Select time, date_format(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo'), 'yyyy-MM-dd HH:mm:ss') as current_date_time, count(*)
from MyTable
where time <= date_format(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo'), 'yyyy-MM-dd HH:mm:ss') 
``


## Formatando valores monetários

Só trocando o ponto por virgula:

```python
df.withColumn('Valor', regexp_replace(col('Valor').cast("string"), '\\.', '\\,'))
```

Formatação completa "europeia" (1.000,00)

```python
df.withColumn("european_format", regexp_replace(regexp_replace(regexp_replace(
            format_number(col("column").cast("double"), 2), '\\.', '#'), ',', '\\.'), '#', ','))
```

## Deduplicando dados com garantia de pegar o mais recente


Jeito "errado": Usando o Rank, se houver empate (duas linhas com o mesmo timestamp) cada um terá o rank igual a 1, ou seja, ainda teremos duplicidades.

```python
my_path = "/path/to?/data"
window = Window.partitionBy(["ID", "ID_ITEM"]).orderBy(col("TS_UTC_INGENTION").desc())
df = spark.read.csv(my_path) \
      .withColumn("rank", rank().over(window)).filter(col("rank") == 1).drop("rank")
```

Jeito "certo":  usar a função _row_number_ que retornará uma sequencia numérica para cada bloco da clausula window

```python
Usar "row_number" Para ter certeza que vai funcionar quando tiver empate
df2 = df.withColumn("rank_temp", row_number().over(window)).filter(col("rank_temp") == 1).drop("rank_temp")
```

## Criando cache de um dataset

É util quando se deseja usar o mesmo dataset varias vezes 

Ver [essa explicação](https://sparkbyexamples.com/spark/spark-dataframe-cache-and-persist-explained/)

```python
 
df = ...

# Criando estado de dataframe na memoria (cache) 
df.cache().count()

# ... Operações diversas nesse dataframe

# REALIZANDO UNPERSIST DO DATAFRAME
df.unpersist()
...


```

## Eliminando todo o cache

Parece que em alguns casos o spark gera novos caches quando você utiliza um dataset colocado em cache, o que pode gerar problemas de memória em aplicações mais pesadas. A função "unpersist" funciona para apagar o cache original mas não resolve dos caches criados automaticamente.

Para uma explicação melhor [ver esse post](https://stackoverflow.com/questions/36905717/un-persisting-all-dataframes-in-pyspark#36909862)


```python
from pyspark import SparkContext, HiveContext

spark_context = SparkContext(appName='cache_test')
hive_context = HiveContext(spark_context)

df = (hive_context.read
      .format('com.databricks.spark.csv')
      .load('simple_data.csv')
     )
df.cache()
df.show()

df = df.withColumn('C1+C2', df['C1'] + df['C2'])
df.cache()
df.show()

spark_context.stop()
```


Solução:

Spark 2.x: Usar Catalog.clearCache:
```python
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate
# ...
spark.catalog.clearCache()
```

Spark 1.x

You can use SQLContext.clearCache method which

Removes all cached tables from the in-memory cache.

```python
from pyspark.sql import SQLContext
from pyspark import SparkContext
sqlContext = SQLContext.getOrCreate(SparkContext.getOrCreate())
#...
sqlContext.clearCache()
...


